# 1. ライブラリのインポートとデータ読み込み

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb  # XGBoostをインポート
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォントの設定
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# 前処理済みのデータを読み込む
df = pd.read_csv('data/suumo_data_cleaned.csv')

df.head()

# 2. モデルの学習と評価

In [ ]:
# 特徴量 (X) とターゲット (y) を定義
# rent_log をターゲットとし、元のrentは含めない
X = df.drop(['rent', 'rent_log'], axis=1)
y = df['rent_log']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'訓練データのサイズ: {X_train.shape}')
print(f'テストデータのサイズ: {X_test.shape}')

In [ ]:
# LightGBMモデルの学習
lgb_reg = lgb.LGBMRegressor(random_state=42)
lgb_reg.fit(X_train, y_train)

# テストデータで予測
y_pred_log_lgb = lgb_reg.predict(X_test)

# 予測結果を元のスケールに戻す (log1pの逆変換はexpm1)
y_pred_lgb = np.expm1(y_pred_log_lgb)
y_test_orig = np.expm1(y_test)

# RMSE (Root Mean Squared Error) で評価
rmse_lgb = np.sqrt(mean_squared_error(y_test_orig, y_pred_lgb))
print(f'LightGBM RMSE: {rmse_lgb:.4f} (万円)')

# 3. 結果の可視化

In [ ]:
# 特徴量の重要度を可視化
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_reg.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('特徴量の重要度 (LightGBM)')
plt.show()

In [ ]:
# 実際の値と予測値の散布図
plt.figure(figsize=(10, 10))
plt.scatter(y_test_orig, y_pred_lgb, alpha=0.5)
plt.plot([y_test_orig.min(), y_test_orig.max()], [y_test_orig.min(), y_test_orig.max()], '--', color='red', linewidth=2)
plt.xlabel('実際の家賃 (万円)')
plt.ylabel('予測家賃 (万円)')
plt.title('実際の値 vs 予測値 (LightGBM)')
plt.grid(True)
plt.show()

In [42]:
# データセット全体の家賃を予測
all_pred_log = lgb_reg.predict(X)
all_pred = np.expm1(all_pred_log)

# 元のDataFrame (df_cleaned) に予測結果を追加
df['predicted_rent'] = all_pred

# 実際の家賃と予測家賃の差額を計算
df['difference'] = df['rent'] - df['predicted_rent']

# 割安度を計算（差額 / 実際の家賃）
# 予測より安い物件はマイナスになる
df['discount_rate'] = df['difference'] / df['rent']

# 元の生データ（建物名など）を読み込み
df_raw_original = pd.read_csv("data/suumo_data.csv")

# 必要な列を結合して新しいDataFrameを構築
# df と df_raw_original は同じ順序であると仮定
df_bargain = pd.DataFrame({
    "building_name": df_raw_original["building_name"],
    "address": df_raw_original["address"],
    "age": df_raw_original["age"], 
    "walk_minutes": df["walk_minutes"], 
    "area": df["area_m2"], 
    "layout": df_raw_original["layout"], 
    "rent": df["rent"], 
    "predicted_rent": df["predicted_rent"],
    "difference": df["difference"],
    "discount_rate": df["discount_rate"]
})

# 割安度の低い順（お得な物件順）にソート
df_bargain = df_bargain.sort_values("discount_rate", ascending=True)

# 結果の表示
print("割安物件ランキング TOP 20 (LightGBM) " )
display(df_bargain[[
    'building_name', 'address', 'age', 'walk_minutes', 'area', 'layout', 
    'rent', 'predicted_rent', 'difference', 'discount_rate'
]].head(20))

割安物件ランキング TOP 20 (LightGBM) 


,building_name,address,age,walk_minutes,area,layout,rent,predicted_rent,difference,discount_rate
199,パークノヴァ九段,東京都千代田区九段南４,築36年,4.0,46.08,ワンルーム,15.5,27.322184,-11.822184,-0.762722
709,プラウド千代田淡路町,東京都千代田区神田須田町１,築9年,2.0,40.03,1LDK,22.0,33.040791,-11.040791,-0.501854
209,番町パークハウス,東京都千代田区四番町,築19年,3.0,133.68,3LDK,80.0,118.718279,-38.718279,-0.483978
210,東京メトロ半蔵門線 半蔵門駅 地下1地上12階建 築9年,東京都千代田区一番町,築9年,3.0,121.85,2LDK,96.0,128.574693,-32.574693,-0.339320
451,ＪＲ総武線 市ケ谷駅 地下1地上14階建 築3年,東京都千代田区四番町,築3年,6.0,72.67,3LDK,55.0,73.479540,-18.479540,-0.335992
450,ＪＲ総武線 市ケ谷駅 地下1地上14階建 築3年,東京都千代田区四番町,築3年,6.0,72.67,3LDK,55.0,73.479540,-18.479540,-0.335992
218,ＪＲ山手線 神田駅 14階建 築20年,東京都千代田区神田錦町１,築20年,9.0,29.98,ワンルーム,12.4,15.726262,-3.326262,-0.268247
382,東京メトロ銀座線 赤坂見附駅 地下3地上38階建 築23年,東京都千代田区永田町２,築23年,1.0,140.17,2LDK,91.9,115.095149,-23.195149,-0.252396
18,プルデンシャルタワーレジデンス,東京都千代田区永田町２,築23年,1.0,140.17,2LDK,91.9,115.095149,-23.195149,-0.252396
198,パークノヴァ九段,東京都千代田区九段南４,築36年,4.0,32.64,ワンルーム,11.6,14.202152,-2.602152,-0.224323


# 5. XGBoostモデルでの検証と性能比較

In [43]:
# XGBoostモデルの学習
xgb_reg = xgb.XGBRegressor(random_state=42)
xgb_reg.fit(X_train, y_train)

# テストデータで予測
y_pred_log_xgb = xgb_reg.predict(X_test)

# 予測結果を元のスケールに戻す
y_pred_xgb = np.expm1(y_pred_log_xgb)

# RMSEで評価
rmse_xgb = np.sqrt(mean_squared_error(y_test_orig, y_pred_xgb))
print(f'XGBoost RMSE: {rmse_xgb:.4f} (万円)')

XGBoost RMSE: 5.7253 (万円)


In [44]:
# モデルの性能比較
print(f'LightGBM RMSE: {rmse_lgb:.4f}')
print(f'XGBoost RMSE : {rmse_xgb:.4f}')

LightGBM RMSE: 4.7160
XGBoost RMSE : 5.7253
